### 문제 3-1 :  콘텐츠분쟁해결 RAG 시스템 - 간단 실습 가이드

In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

loader = PyPDFLoader("../data/콘텐츠분쟁해결_사례.pdf")
documents = loader.load()  
print(f"  총 {len(documents)}페이지 로드 완료")

print("==> 1단계: 문서 분할 설정")
pythontext_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=300,
    separators=[
        "\n【사건개요】", "\n【쟁점사항】", "\n【처리경위】", "\n【처리결과】",
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)
chunks = pythontext_splitter.split_documents(documents)

print(f"  {len(chunks)}개 청크 생성 완료")
print(f"  평균 청크 길이: {sum(len(chunk.page_content) for chunk in chunks) / len(chunks):.0f}자")

print("==> 2단계: 임베딩 모델 설정")
pythonembeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    dimensions=1536
)

vectorstore = FAISS.from_documents(chunks, pythonembeddings)

print("===> 3단계: 검색기 설정")
pythonretriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)
print(" Retriever 설정 완료")

print("===> 4단계: LLM 설정")
pythonllm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.2,
    max_tokens=2000
)

print("===> 5단계: 법률 자문 프롬프트 작성")
pythonprompt_template = PromptTemplate.from_template("""
당신은 콘텐츠 분야 전문 법률 자문사입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요
2. 관련 법령이나 조항이 있다면 명시하세요
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요
4. 실무적 해결방안을 단계별로 제시하세요
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요

전문 법률 조언:
""")

print("===> 6단계: QA 체인 생성")
pythonqa_chain = RetrievalQA.from_chain_type(
    llm=pythonllm,
    retriever=pythonretriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": pythonprompt_template},
    return_source_documents=True
)

print("===> 7단계: 테스트 질문 작성")
pythontest_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]


for i, question in enumerate(pythontest_questions, 1):
    print(f"\n【테스트 {i}/{len(pythontest_questions)}】")
    print(f" 질문: {question}")
    print(" 답변 생성 중...")

    # RAG 실행
    result = pythonqa_chain.invoke({"query": question})
    answer = result["result"]
    source_docs = result["source_documents"]

    print(f"\n📌 답변:")
    print("-" * 50)
    print(answer)

    # 참조 문서 정보 출력
    print(f"\n📚 참조 문서:")
    for j, doc in enumerate(source_docs[:3], 1):  # 상위 3개만 표시
        page = doc.metadata.get('page', 'N/A')
        preview = doc.page_content[:80].replace('\n', ' ').strip()
        print(f"  {j}. 페이지 {page}: {preview}...")

    print("\n" + "=" * 60)


  총 109페이지 로드 완료
==> 1단계: 문서 분할 설정
  104개 청크 생성 완료
  평균 청크 길이: 722자
==> 2단계: 임베딩 모델 설정
===> 3단계: 검색기 설정
 Retriever 설정 완료
===> 4단계: LLM 설정
===> 5단계: 법률 자문 프롬프트 작성
===> 6단계: QA 체인 생성
===> 7단계: 테스트 질문 작성

【테스트 1/5】
 질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
 답변 생성 중...

📌 답변:
--------------------------------------------------
온라인 게임에서 시스템 오류로 인해 아이템이 사라진 경우, 게임회사가 복구를 거부하는 상황에 대한 법률 조언을 드리겠습니다. 제시된 사례들을 바탕으로 다음과 같은 조언을 드립니다.

1. **계정 명의 문제**:
   - 사례 1-가-1(2006)에서는 계정 명의가 본인이 아닌 경우 복구가 불가하다고 판단되었습니다. 따라서, 계정 명의가 본인인지 확인하고, 본인이 아닐 경우 계정 명의자가 직접 복구 요청을 해야 합니다.

2. **시스템 오류 여부 확인**:
   - 사례 1-가-1(2009)에서는 시스템 오류로 인한 손실을 주장했으나, 다른 사용자들에게 유사한 불만이 접수되지 않았고, 사용 내역상 문제가 없었다고 판단되어 복구가 거부되었습니다. 따라서, 시스템 오류가 실제로 발생했는지, 다른 사용자들에게도 유사한 문제가 있었는지 확인하는 것이 중요합니다.

3. **보안 서비스 사용 여부**:
   - 사례 1-가-1(2009)에서는 보안 서비스를 충분히 사용하지 않아 해킹 피해에 대한 복구가 거부되었습니다. 따라서, 게임 회사가 제공하는 보안 서비스를 충분히 사용했는지 확인해야 합니다.

4. **법적 근거**:
   - 민법 제250조와 제251조에 따르면, 도품이나 유실물의 경우 피해자는 일정 기간 내에 반환을 청구할 수 있습니다. 그